In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
import os

# Input the OpenAI API key
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

In [ ]:
reader = SimpleDirectoryReader("./data/")
documents = reader.load_data()

In [ ]:
documents

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

nodes = SentenceSplitter().get_nodes_from_documents(documents)

In [ ]:
print(f"There are total {len(nodes)} nodes")

In [ ]:
from app.integrations.llama_index.docstore.mssql import MSSQLDocumentStore

from app.settings import Constants, Secrets

docstore = MSSQLDocumentStore.from_uri(
    uri=Constants.MSSQL_CONNECTOR_URI.format(
        user=Secrets.MSSQL_USER,
        password=Secrets.MSSQL_SA_PASSWORD,
        host="127.0.0.1",
        port=Secrets.MSSQL_PORT,
        db_name=Secrets.MSSQL_DB,
        driver=Constants.MSSQL_DRIVER
    )
)
docstore.add_documents(nodes)

In [ ]:
from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults(docstore=docstore)

In [ ]:
len(storage_context.docstore.docs)

In [ ]:
llm = OpenAI(temperature=0, model="gpt-4o-mini")

Settings.llm = llm
Settings.chunk_size = 1024

In [ ]:
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

In [ ]:
query_engine = vector_index.as_query_engine()
response = query_engine.query("What is a summary of this document?")

In [ ]:
print(response)